In [1]:
# imports 
import requests
import pandas as pd 

# get url and set
# get desired sinopia platform for upload
sinopia_platform = ""
file_name = ""
print_name = ""

platform_prompt = """Select a Sinopia Environment to see resources from: 
\n[1] Development\n[2] Stage\n[3] Production\n> """
platform = input(platform_prompt)
if platform == "1":
    sinopia_platform = "development."
    file_name = "sinopia_dev"
    print_name = "Sinopia Development"
    print(print_name + " selected")
elif platform == "2":
    sinopia_platform = "stage."
    file_name = "sinopia_stage"
    print_name = "Sinopia Stage"
    print(print_name + " selected")
elif platform == "3":
    sinopia_platform = ""
    file_name = "sinopia"
    print_name = "Sinopia Production"
    print(print_name + " selected")
else:
    print("Platform not recognized.")
    exit(0)

url = "https://api." + sinopia_platform + "sinopia.io/resource/"

Sinopia Stage selected


In [2]:
#GET HTTP request
get_sinopia = requests.get(url,{"group": "washington", "limit": 1000})

# if 200 - good to go 
status_code = get_sinopia.status_code
print(status_code)

#get results - NOTE: limit is 1000 
resources = get_sinopia.json()

200


In [3]:
#display results

#variables 
label_key = [
    "http://id.loc.gov/ontologies/bibframe/mainTitle",
    "http://rdaregistry.info/Elements/w/P10223",
    "http://rdaregistry.info/Elements/e/P20315",
    "http://rdaregistry.info/Elements/i/P40085",
    "http://rdaregistry.info/Elements/m/P30156",
    "http://rdaregistry.info/Elements/n/P80068"
    ]
num = len(resources["data"])
resource_list = []

# retrieve resources (not rts)
i = 0
while i < num: 
    type = resources["data"][i]["types"]
    if (type[0] != 'http://sinopia.io/vocabulary/ResourceTemplate'):
        label = "None"
        data_len = len(resources["data"][i]["data"])
        j = 0
        while j < data_len:
            k = 0
            # retrieve label if exists
            while k < len(label_key):
                if label_key[k] in resources["data"][i]["data"][j]:
                    if "@value" in resources["data"][i]["data"][j][label_key[k]][0]:
                        label = resources["data"][i]["data"][j][label_key[k]][0]["@value"]
                k = k + 1
            j = j + 1
        
        # store label, uri, templateId, and timestamp in dict
        resource_dict = {'Label':label, 'URI':resources["data"][i]["uri"], 'TemplateId':resources["data"][i]["templateId"],
                         'Timestamp':resources["data"][i]["timestamp"]}
        # add dict to resource list
        resource_list.append(resource_dict)
    i = i + 1

# sort list by most recent
resource_list = sorted(resource_list, key=lambda x: x['Timestamp'], reverse=True)
# create dataframe from list and display
resource_df = pd.DataFrame.from_dict(resource_list)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
resource_df


,Label,URI,TemplateId,Timestamp
0,None,https://api.stage.sinopia.io/resource/4d0ea5cf-af6b-4f2c-a75b-96a9bdee9bfa,UWSINOPIA:WAU:rdaWork:metadataWork:CAMS,2023-06-30T03:23:44.876Z
1,Labels for locals,https://api.stage.sinopia.io/resource/33a564c1-c1f5-4303-8182-c2c6abb27ccc,UWSINOPIA:WAU:rdaExpression:printMonograph:CAMS,2023-06-29T19:18:42.215Z
2,Labels for locals,https://api.stage.sinopia.io/resource/82f2c36c-7213-45da-8fcc-9e820ffeb597,UWSINOPIA:WAU:rdaWork:printMonograph:CAMS,2023-06-29T19:17:37.728Z
3,"Elden, Stuart, 1971- The archaeology of Foucault",https://api.stage.sinopia.io/resource/6bb8a297-1717-4b85-beb1-9c4866c9210a,UWSINOPIA:WAU:rdaNomen:accessPoint:CAMS,2023-06-13T21:15:04.544Z
4,The archaeology of Foucault,https://api.stage.sinopia.io/resource/0d8c81b7-4fd6-4cc7-959e-6e44909e5b4d,UWSINOPIA:WAU:rdaManifestation:printMonograph:CAMS,2023-06-13T21:14:52.839Z
5,The archaeology of Foucault,https://api.stage.sinopia.io/resource/70f1602d-d21e-483f-80f4-0d684944527a,UWSINOPIA:WAU:rdaExpression:printMonograph:CAMS,2023-06-13T21:12:44.024Z
6,The archaeology of Foucault,https://api.stage.sinopia.io/resource/f4b14d86-95f4-4767-8430-699e4298e454,UWSINOPIA:WAU:rdaWork:printMonograph:CAMS,2023-06-13T21:07:21.512Z
7,The archaeology of Foucault,https://api.stage.sinopia.io/resource/e5a19e15-610f-400e-bea1-1b04cc922288,UWSINOPIA:WAU:rdaItem:printMonograph:CAMS,2023-06-13T21:00:31.265Z
8,Japan as viewed by 17 creators,https://api.stage.sinopia.io/resource/43a1bf89-5c3f-44ca-ae36-6bd34f352b78,UWSINOPIA:WAU:rdaWork:aggregating_printMonograph:CAMS,2023-05-12T05:48:02.440Z
9,Japan as viewed by 17 creators,https://api.stage.sinopia.io/resource/eb9957b7-0ce3-4927-a8ba-9590bdfd5174,UWSINOPIA:WAU:rdaManifestation:printMonograph:CAMS,2023-05-12T05:47:42.673Z


In [10]:
# can save as excel sheet
xlsx_file = file_name + "_resource_list.xlsx"
writer = pd.ExcelWriter(xlsx_file, engine="xlsxwriter")

resource_df.to_excel(writer, sheet_name="Sheet1", startrow=1, header=False, index=False)

workbook = writer.book
worksheet = writer.sheets["Sheet1"]
(max_row, max_col) = resource_df.shape
column_settings = [{"header": column} for column in resource_df.columns]
worksheet.add_table(0, 0, max_row, max_col - 1, {"columns": column_settings, 'style': 'Table Style Light 12'})
worksheet.set_column(0, 0, 20)
worksheet.set_column(1, 1, 70)
worksheet.set_column(2, 2, 57)
worksheet.set_column(3, 3, 24)

writer.close()